In [1]:
# 1. Imports:
import pandas as pd
import requests
from shapely.geometry import Point
import geopandas as gpd

In [2]:
# 2. Acquisition & Wrangling:

In [3]:
# 2.a) Read & transform csv

In [4]:
PATH = '../data/bicimad_stations.csv'
GEO = 'geometry.coordinates'

In [5]:
def read_csv(file_path):
    df_read_csv = pd.read_csv(PATH, sep='\t', index_col=0)
    return df_read_csv

In [6]:
read_csv_output=read_csv(PATH)

In [7]:
def string_to_coordinates(string):
    list_coordinates = [float(element) for element in string.replace("[", "").replace("]", "").replace(" ","").split(",")]
    return list_coordinates

In [8]:
def normalize_csv(df, orig_column, new_col1, new_col2):
    df[orig_column] = df[orig_column].apply(lambda row: string_to_coordinates(row))
    df[new_col1] = df[orig_column].apply(lambda row: row[0])
    df[new_col2] = df[orig_column].apply(lambda row: row[1])
    return df

In [9]:
df_bicimad=normalize_csv(read_csv_output, GEO, 'bm_longitude', 'bm_latitude')
df_bicimad.head()

,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates,bm_longitude,bm_latitude
0,1,1a - Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.7018341, 40.4172137]",-3.701834,40.417214
1,2,1b - Puerta del Sol B,3,1b,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.701602938060457, 40.41731271011562]",-3.701603,40.417313
2,3,2 - Miguel Moya,3,2,Calle Miguel Moya nº 1,1,1,24,0,0,0,Point,"[-3.7058415, 40.4205886]",-3.705842,40.420589
3,4,3 - Plaza Conde Suchil,2,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,9,9,0,Point,"[-3.7069171, 40.4302937]",-3.706917,40.430294
4,5,4 - Malasaña,1,4,Calle Manuela Malasaña nº 5,1,0,24,23,1,0,Point,"[-3.7025875, 40.4285524]",-3.702587,40.428552


In [10]:
# 2.b) Get data from API REST and transform

In [11]:
API_ENDPOINT="https://datos.madrid.es/egob"
DATASET="/catalogo/300356-0-monumentos-ciudad-madrid.json"
COLUMNS_2FIX = ["address", "location", "organization"] 

In [12]:
def get_dataset(api_endpoint, dataset):
    response = requests.get(api_endpoint+dataset)
    json_data = response.json() 
    df_dataset = pd.DataFrame(json_data["@graph"]) # the data that we need is stored in key "@graph" within json
    return df_dataset

In [13]:
get_dataset_output = get_dataset(API_ENDPOINT, DATASET)

In [14]:
def normalize_dataset(df):
    df = df.dropna() # 1 drop nulls
    for column in COLUMNS_2FIX: # 2 normalize dictionaries within columns
        df = pd.concat([df.drop(columns = [column]), df[column].apply(lambda x: pd.Series(x))], axis = 1)
    return df

In [15]:
df_monuments = normalize_dataset(get_dataset_output)
#df_monuments.info()

In [16]:
# 2.c) Build main df as cross join of bicimad stations and places of interest

In [17]:
def merge(df1, df2):
    df_merged = pd.merge(df1, df2, how="cross")
    return df_merged

In [18]:
full_dataset = merge(df_monuments, df_bicimad)

In [30]:
df_sample = full_dataset.loc[:791, :].copy()

In [31]:
# 3. Analysis:

In [32]:
# 3.a) Calculate the distance to each bicimad station

In [46]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair points in degrees 
    # (e.g.: Start Point -> 40.4400607 / -3.6425358 End Point -> 40.4234825 / -3.6292625)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)

In [47]:
def add_distance_col(df, col_name, lat_start, long_start, lat_finish, long_finish):
    df[col_name] = df.apply(lambda row: distance_meters(row[lat_start], row[long_start],
                                                                          row[lat_finish], row[long_finish]), axis = 1)
    return df

In [48]:
df_sample = add_distance_col(df_sample, "distance", "latitude", "longitude", "bm_latitude", "bm_longitude")

In [49]:
df_sample.head()

,@id,id_x,title,relation,references,district,locality,postal-code,street-address,area,...,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates,bm_longitude,bm_latitude,distance
0,https://datos.madrid.es/egob/catalogo/tipo/mon...,409634,A las víctimas del Holocausto,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,{'@id': 'https://datos.madrid.es/egob/kos/Prov...,MADRID,,JARDIN TRES CULTURAS,NaN,...,1,30,0,0,0,Point,"[-3.7018341, 40.4172137]",-3.701834,40.417214,12152.077352
1,https://datos.madrid.es/egob/catalogo/tipo/mon...,409634,A las víctimas del Holocausto,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,{'@id': 'https://datos.madrid.es/egob/kos/Prov...,MADRID,,JARDIN TRES CULTURAS,NaN,...,1,30,0,0,0,Point,"[-3.701602938060457, 40.41731271011562]",-3.701603,40.417313,12124.059117
2,https://datos.madrid.es/egob/catalogo/tipo/mon...,409634,A las víctimas del Holocausto,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,{'@id': 'https://datos.madrid.es/egob/kos/Prov...,MADRID,,JARDIN TRES CULTURAS,NaN,...,1,24,0,0,0,Point,"[-3.7058415, 40.4205886]",-3.705842,40.420589,12404.627842
3,https://datos.madrid.es/egob/catalogo/tipo/mon...,409634,A las víctimas del Holocausto,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,{'@id': 'https://datos.madrid.es/egob/kos/Prov...,MADRID,,JARDIN TRES CULTURAS,NaN,...,0,18,9,9,0,Point,"[-3.7069171, 40.4302937]",-3.706917,40.430294,12135.695695
4,https://datos.madrid.es/egob/catalogo/tipo/mon...,409634,A las víctimas del Holocausto,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,{'@id': 'https://datos.madrid.es/egob/kos/Prov...,MADRID,,JARDIN TRES CULTURAS,NaN,...,0,24,23,1,0,Point,"[-3.7025875, 40.4285524]",-3.702587,40.428552,11742.130030


In [50]:
# 3.b) Group by place of interest:

In [51]:
def group_by(df, col_group, col_agg):
    df_grouped = df.groupby([col_group]).agg(min_value = (col_agg, 'min'), idxmin = (col_agg, 'idxmin')).reset_index()
    df_grouped['idxmin'] = df_grouped['idxmin'].map(df['id_y'])
    df_grouped = df_grouped.rename(columns = {'idxmin': 'id_y', 'min_value': col_agg})
    return df_grouped


In [52]:
group_by_output = group_by(df_sample, "id_x", "distance")
group_by_output.head()

,id_x,distance,id_y
0,409268,94.201295,99
1,409512,2633.107767,257
2,409634,5559.887276,238


In [53]:
# 3.c) Build final table with all necesary columns and create csv

In [54]:
TABLE_COLS = ['Place of Interest', 'Type of place', 'Place address', 'BiciMAD station', 'Station locaiton', 'Available bikes']
PLACE = 'Monuments'
FILE_PATH = '../data/nearest_bicimad.csv'

In [55]:
def build_output_table(df_grouped, df_places, df_bici, place, table_cols, file_path):
    
    res1 = df_grouped.merge(df_places[['id', 'title', 'street-address']],
                                how='inner', left_on='id_x', right_on='id').drop(columns = ['id']) # add place columns
    
    res2 = res1.merge(df_bici[['id', 'name', 'address', 'dock_bikes']],
                                how='inner', left_on='id_y', right_on='id').drop(columns = ['id']) # add bicimad columns

    res3 = res2.drop(columns = ['distance', 'id_x', 'id_y']) # drop ids


    res3['Type of place'] = place # add column identifying type of place

    #  move new column to the right position
    cols = list(res3.columns.values)
    cols = cols[:1] + cols[-1:] + cols[1:-1]
    res3 = res3[cols] 

    res3.columns = table_cols # rename columns
    
    res3.to_csv(file_path, index=False) # create csv
        
    return  res3

In [56]:
output_table = build_output_table(group_by_output, df_monuments, df_bicimad, PLACE, TABLE_COLS, FILE_PATH)

In [57]:
output_table

,Place of Interest,Type of place,Place address,BiciMAD station,Station locaiton,Available bikes
0,"A Marca, promotor de la San Silvestre Vallecana",Monuments,Pº Recoletos,94 - Biblioteca Nacional,Paseo de Recoletos nº 20,15
1,A los abuelos y abuelas,Monuments,C Alicún,249 - Manuel Caldeiro,Paseo de la Castellana nº 298,11
2,A las víctimas del Holocausto,Monuments,JARDIN TRES CULTURAS,230 - Carlota O’Neill,Calle Carlota O'Neill nº 32,13


In [ ]:
# We want to give user the possibility to access only one line in the output table:

In [63]:
place = input("Por favor, introduzca el lugar de interés: ")
print(output_table.loc[output_table['Place of Interest'].isin([place])])
# mi estrategia va a ser:
# - si ejecutamos desde la terminal main.py --> crea el csv con todos los datos
# - si ejemuctamos desde la terminal -f main.py --> pide al usuario elegir el sitio de interés y muestra la tabla directamente

Por favor, introduzca el lugar de interés: A los abuelos y abuelas
         Place of Interest Type of place Place address        BiciMAD station  \
1  A los abuelos y abuelas     Monuments     C Alicún   249 - Manuel Caldeiro   

                Station locaiton  Available bikes  
1  Paseo de la Castellana nº 298               11  


In [58]:
DF_CSV = read_csv(PATH)
DF_DATASET = get_dataset(API_ENDPOINT, DATASET)
DF_BICIMAD = normalize_csv(DF_CSV, GEO, 'bm_longitude', 'bm_latitude')
DF_MONUMENTS = normalize_dataset(DF_DATASET)
DF_FULL_DATASET = merge(DF_MONUMENTS, DF_BICIMAD)
DF_SAMPLE = DF_FULL_DATASET.loc[:791, :].copy() # we create a sample to shorten execution time in the next step
DF_DISTANCE = add_distance_col(DF_SAMPLE, "distance", "latitude", "longitude", "bm_latitude", "bm_longitude")
DF_GROUPED = group_by(DF_DISTANCE, "id_x", "distance")
DF_OUTPUT = build_output_table(DF_GROUPED, DF_MONUMENTS, DF_BICIMAD, PLACE, TABLE_COLS, FILE_PATH)